In [1]:
import os
import re
import numpy as np
import pandas as pd
import datetime

import matplotlib.pyplot as plt
import seaborn as sns

import scorpyo as sp

from null_importance import get_null_importance


pd.set_option('max_rows', 500, 'max_columns', 200)

In [2]:
path_project = r'/Users/liliangshan/workspace/python/01_datasets/ccf_system_access_risk_identification'

# path dir
path_row_data = os.path.join(path_project, 'row_data')
path_new_data = os.path.join(path_project, 'new_data')
path_results  = os.path.join(path_project, 'results')
path_results_jupyter  = os.path.join(path_results, 'jupyter')

# path row_data
path_train = os.path.join(path_row_data, 'train.csv')
path_test  = os.path.join(path_row_data, 'evaluation_public.csv')
path_sample_submission = os.path.join(path_row_data, 'submit_example.csv')


path_new_train = os.path.join(path_new_data, 'train_lightgbm_20221014.csv')
path_new_test  = os.path.join(path_new_data, 'test_lightgbm_20221014.csv')

## results
path_output_report = os.path.join(path_results, '01_原始数据探察_20221014.xlsx')

y_label = "is_risk"

In [3]:
df_row_train = sp.read_data(path_train)
df_row_val  = sp.read_data(path_test)

df_row_train['url_sit'] = df_row_train['url'].map(lambda x: x.split('/')[0])
df_row_train['url_page'] = df_row_train['url'].map(lambda x: x.split('/')[1])

df_row_val['url_sit'] = df_row_val['url'].map(lambda x: x.split('/')[0])
df_row_val['url_page'] = df_row_val['url'].map(lambda x: x.split('/')[1])



df_train_info = df_row_train.copy().sort_values(by='op_datetime')

## 定义一次/一天/一月
1. op\_times\_groups 一次
2. op\_days 一天
3. op\_month 一月

In [4]:
df = pd.concat([df_row_train, df_row_val]).reset_index(drop=True)
df = df.sort_values(by='op_datetime')

# 认证日期时间
df['op_datetime'] = pd.to_datetime(df['op_datetime'])
# 将数据分为每一天
df['op_days'] = df['op_datetime'].map(lambda x: x.strftime('%Y-%m-%d'))

# 按客户进行统计这次认证和上次认证的时间差
df['op_second'] = df['op_datetime']
df['op_second1'] = df.groupby('device_num_transform')['op_second'].shift(1)
df['op_diff_second1'] = (df['op_second'] - df['op_second1']).map(lambda x: x.total_seconds())
df['op_diff_second1_tmp'] = df['op_diff_second1']>10
# 客户第几次登录
df['op_times_groups'] = df.groupby('device_num_transform')['op_diff_second1_tmp'].apply(lambda x: x.cumsum())

# 系统层面的一段时间
df['system_op_second'] = df['op_datetime']
df['system_op_second1'] = df['system_op_second'].shift(1)
df['system_op_diff_second1'] = (df['system_op_second'] - df['system_op_second1']).map(lambda x: x.total_seconds())
df['system_op_diff_second1_tmp'] = df['system_op_diff_second1']>400
# 客户第几次登录
df['system_op_times_groups'] = df['system_op_diff_second1_tmp'].cumsum()

df = df.drop(columns=['op_second','op_second1','op_diff_second1_tmp', 
                      'system_op_second', 'system_op_second1', 'system_op_diff_second1_tmp'])

## 环境特征

In [5]:

time_feats = ['system_op_times_groups', 'op_days', 'op_month']

cate_feats = ['ip_transform', 'user_name', 'device_num_transform', 'department', 'browser_version', 'browser', 'os_type','os_version',
              'ip_type','http_status_code', 'op_city', 'log_system_transform', 'url', 'url_page','url_sit']

# 客户+时间+按时间cumsum/cumunique
# 累计量统计
df['helper'] = 1
# 是否为异常状态码
df['http_status_code_helper'] = df['http_status_code'].map(lambda x: 1 if x in [400, 500, 502, 404] else 0) 
# 给样本编号
df['sampler_index_helper'] = df['helper'].cumsum()

for i in time_feats:
    i_tmp = df.groupby([i])
    # 系统往前看，处理了多少事
    df['system_{}_cumsum'.format(i)] = i_tmp['helper'].cumsum()
    # 系统往前看，处理了多少坏事
    df['system_{}_error_code_cumsum'.format(i)] = i_tmp['http_status_code_helper'].cumsum()
    for j in cate_feats:
        index_set = set(df.groupby([i, j],as_index=False).first()['sampler_index_helper'].values)
        df['tmp_helper'] = df['sampler_index_helper'].map(lambda x: 1 if x in index_set else 0)
        # 系统往前看，不同维度的处理了多少情况
        df['system_{}_{}_cumunique'.format(i, j)] = df.groupby([i, j])['tmp_helper'].cumsum()
        
        if j not in ['ip_transform', 'user_name', 'device_num_transform']:
            for k in df[j].unique():
                tmp = df[df[j]==k].groupby([i])

                # 系统往前看，不同维度不同情况分别处理了多少次
                df['system_{}_{}_{}_cumsum'.format(i,j,k)] = tmp['helper'].cumsum()
                # 系统往前看，不同维度不同情况error_code分别处理了多少次
                df['system_{}_{}_{}_error_code_cumsum'.format(i,j,k)] = tmp['http_status_code_helper'].cumsum()

remove_cols = [x for x in df.columns if x[-6:]=='helper']

df = df.drop(columns=remove_cols)


## 对象层面

In [6]:

time_feats = ['op_times_groups', 'op_days', 'op_month']

object_feats = ['ip_transform', 'user_name', 'device_num_transform']

cate_feats = ['ip_transform', 'user_name', 'device_num_transform', 'department', 'browser_version', 'browser', 'os_type','os_version',
              'ip_type','http_status_code', 'op_city', 'log_system_transform', 'url', 'url_page','url_sit']

# 客户+时间+按时间cumsum/cumunique
# 累计量统计
df['helper'] = 1
# 是否为异常状态码
df['http_status_code_helper'] = df['http_status_code'].map(lambda x: 1 if x in [400, 500, 502, 404] else 0) 
# 给样本编号
df['sampler_index_helper'] = df['helper'].cumsum()


for i in time_feats:
    for j in object_feats:
        j_tmp = df.groupby([i,j])
        df['{}_{}_cumsum'.format(i,j)] = j_tmp['helper'].cumsum()
        df['{}_{}_error_code_cumsum'.format(i,j)] = j_tmp['http_status_code_helper'].cumsum()
        for k in cate_feats:
            if k == j: continue
            index_set = set(df.groupby([i,j,k], as_index=False).first()['sampler_index_helper'].values)            
            df['tmp_helper'] = df['sampler_index_helper'].map(lambda x: 1 if x in index_set else 0)
            df['{}_{}_{}_cumunique'.format(i,j,k)] = df.groupby([i,j,k])['tmp_helper'].cumsum()
            
            if j not in ['ip_transform', 'user_name', 'device_num_transform']:
                for v in df[k].unique():
                    v_tmp = df[df[k]==v].groupby([i,j])
                    df['{}_{}_{}_{}_cumsum'.format(i,j,k,v)] = v_tmp['helper'].cumsum()
                    df['{}_{}_{}_{}_error_code_cumsum'.format(i,j,k,v)] = v_tmp['http_status_code_helper'].cumsum()

remove_cols = [x for x in df.columns if x[-6:]=='helper']

df = df.drop(columns=remove_cols)

## 其它操作

In [7]:
# 几点钟
df['hour'] = df['op_datetime'].dt.hour
df['hour_sin'] = np.sin(df['hour']/24*2*np.pi)
df['hour_cos'] = np.cos(df['hour']/24*2*np.pi)

# 周几
df['dayofweek'] = df['op_datetime'].dt.dayofweek
df['dayofweek_sin'] = np.sin(df['dayofweek']/7*2*np.pi)
df['dayofweek_cos'] = np.cos(df['dayofweek']/7*2*np.pi)

# 一个月的第几天
df['day'] = df['op_datetime'].dt.day
df['day_sin'] = np.sin(df['day']/31*2*np.pi)
df['day_cos'] = np.cos(df['day']/31*2*np.pi)

## 特征筛选

In [8]:
# 删除时间及类别型变量过多的特征
remove_col = ['op_datetime', 'op_month', 'user_name', 'ip_transform', 'device_num_transform', 'op_days', 'ts', 'ts1', 'ts2','ts3']

remove_col = [x for x in df.columns if x in remove_col]
df = df.drop(columns=remove_col)
df.head()

,id,department,browser_version,browser,os_type,os_version,ip_type,http_status_code,op_city,log_system_transform,url,is_risk,url_sit,url_page,op_diff_second1,op_times_groups,system_op_diff_second1,system_op_times_groups,system_system_op_times_groups_cumsum,system_system_op_times_groups_error_code_cumsum,system_system_op_times_groups_ip_transform_cumunique,system_system_op_times_groups_user_name_cumunique,system_system_op_times_groups_device_num_transform_cumunique,system_system_op_times_groups_department_cumunique,system_system_op_times_groups_department_rd_cumsum,system_system_op_times_groups_department_rd_error_code_cumsum,system_system_op_times_groups_department_hr_cumsum,system_system_op_times_groups_department_hr_error_code_cumsum,system_system_op_times_groups_department_sales_cumsum,system_system_op_times_groups_department_sales_error_code_cumsum,system_system_op_times_groups_department_other_cumsum,system_system_op_times_groups_department_other_error_code_cumsum,system_system_op_times_groups_department_nan_cumsum,system_system_op_times_groups_department_nan_error_code_cumsum,system_system_op_times_groups_department_accounting_cumsum,system_system_op_times_groups_department_accounting_error_code_cumsum,system_system_op_times_groups_browser_version_cumunique,system_system_op_times_groups_browser_version_chrome_93_cumsum,system_system_op_times_groups_browser_version_chrome_93_error_code_cumsum,system_system_op_times_groups_browser_version_safari_13_cumsum,system_system_op_times_groups_browser_version_safari_13_error_code_cumsum,system_system_op_times_groups_browser_version_firefox_78_cumsum,system_system_op_times_groups_browser_version_firefox_78_error_code_cumsum,system_system_op_times_groups_browser_version_edge_93_cumsum,system_system_op_times_groups_browser_version_edge_93_error_code_cumsum,system_system_op_times_groups_browser_version_ie_9_cumsum,system_system_op_times_groups_browser_version_ie_9_error_code_cumsum,system_system_op_times_groups_browser_version_chrome_77_cumsum,system_system_op_times_groups_browser_version_chrome_77_error_code_cumsum,system_system_op_times_groups_browser_version_chrome_90_cumsum,system_system_op_times_groups_browser_version_chrome_90_error_code_cumsum,system_system_op_times_groups_browser_version_ie_11_cumsum,system_system_op_times_groups_browser_version_ie_11_error_code_cumsum,system_system_op_times_groups_browser_cumunique,system_system_op_times_groups_browser_chrome_cumsum,system_system_op_times_groups_browser_chrome_error_code_cumsum,system_system_op_times_groups_browser_safari_cumsum,system_system_op_times_groups_browser_safari_error_code_cumsum,system_system_op_times_groups_browser_firefox_cumsum,system_system_op_times_groups_browser_firefox_error_code_cumsum,system_system_op_times_groups_browser_edge_cumsum,system_system_op_times_groups_browser_edge_error_code_cumsum,system_system_op_times_groups_browser_ie_cumsum,system_system_op_times_groups_browser_ie_error_code_cumsum,system_system_op_times_groups_os_type_cumunique,system_system_op_times_groups_os_type_win_cumsum,system_system_op_times_groups_os_type_win_error_code_cumsum,system_system_op_times_groups_os_type_macos_cumsum,system_system_op_times_groups_os_type_macos_error_code_cumsum,system_system_op_times_groups_os_version_cumunique,system_system_op_times_groups_os_version_win10_cumsum,system_system_op_times_groups_os_version_win10_error_code_cumsum,system_system_op_times_groups_os_version_macos_big_sur_11_cumsum,system_system_op_times_groups_os_version_macos_big_sur_11_error_code_cumsum,system_system_op_times_groups_os_version_win7_cumsum,system_system_op_times_groups_os_version_win7_error_code_cumsum,system_system_op_times_groups_os_version_win11_cumsum,system_system_op_times_groups_os_version_win11_error_code_cumsum,system_system_op_times_groups_ip_type_cumunique,system_system_op_times_groups_ip_type_内网_cumsum,system_system_op_times_groups_ip_type_内网_error_code_cumsum,system_system_op_times_groups_ip_type_nan_cumsum,system_

### 评分卡特征筛选

In [9]:
df_row_train = df[df[y_label].notna()].reset_index(drop=True)
df_row_val = df[df[y_label].isna()].reset_index(drop=True)

df_train, df_test, convert_cols = sp.transform_data_detail(df_row_train, df_row_val, y_label, excel_path=path_output_report)
df_train.head()

sheet05.可能为数值类型的object类型数据统计在/Users/liliangshan/workspace/python/01_datasets/ccf_system_access_risk_identification/results/01_原始数据探察_20221014.xlsx中已经存在，我们将对原文件进行覆盖
sheet06.数据预处理在/Users/liliangshan/workspace/python/01_datasets/ccf_system_access_risk_identification/results/01_原始数据探察_20221014.xlsx中已经存在，我们将对原文件进行覆盖


,id,department,browser_version,browser,os_version,op_city,log_system_transform,url,is_risk,url_sit,url_page,op_diff_second1,op_times_groups,system_op_diff_second1,system_op_times_groups,system_system_op_times_groups_cumsum,system_system_op_times_groups_error_code_cumsum,system_system_op_times_groups_department_rd_cumsum,system_system_op_times_groups_department_rd_error_code_cumsum,system_system_op_times_groups_department_sales_cumsum,system_system_op_times_groups_department_sales_error_code_cumsum,system_system_op_times_groups_browser_version_edge_93_cumsum,system_system_op_times_groups_browser_version_edge_93_error_code_cumsum,system_system_op_times_groups_browser_version_chrome_90_cumsum,system_system_op_times_groups_browser_version_chrome_90_error_code_cumsum,system_system_op_times_groups_browser_chrome_cumsum,system_system_op_times_groups_browser_chrome_error_code_cumsum,system_system_op_times_groups_browser_edge_cumsum,system_system_op_times_groups_browser_edge_error_code_cumsum,system_system_op_times_groups_os_type_win_cumsum,system_system_op_times_groups_os_type_win_error_code_cumsum,system_system_op_times_groups_os_version_win10_cumsum,system_system_op_times_groups_os_version_win10_error_code_cumsum,system_system_op_times_groups_os_version_win7_cumsum,system_system_op_times_groups_os_version_win7_error_code_cumsum,system_system_op_times_groups_ip_type_内网_cumsum,system_system_op_times_groups_ip_type_内网_error_code_cumsum,system_system_op_times_groups_http_status_code_200_cumsum,system_system_op_times_groups_op_city_深圳_cumsum,system_system_op_times_groups_op_city_深圳_error_code_cumsum,system_system_op_times_groups_op_city_成都_cumsum,system_system_op_times_groups_op_city_成都_error_code_cumsum,system_system_op_times_groups_op_city_杭州_cumsum,system_system_op_times_groups_op_city_杭州_error_code_cumsum,system_system_op_times_groups_op_city_北京_cumsum,system_system_op_times_groups_op_city_北京_error_code_cumsum,system_system_op_times_groups_log_system_transform_nHrKgKdJ1Mzt_cumsum,system_system_op_times_groups_log_system_transform_nHrKgKdJ1Mzt_error_code_cumsum,system_system_op_times_groups_log_system_transform_dwS3cdn15GK4_cumsum,system_system_op_times_groups_log_system_transform_dwS3cdn15GK4_error_code_cumsum,system_system_op_times_groups_url_xxx.com/github_cumsum,system_system_op_times_groups_url_xxx.com/github_error_code_cumsum,system_system_op_times_groups_url_wpsdoc.xxx.com/download_cumsum,system_system_op_times_groups_url_wpsdoc.xxx.com/download_error_code_cumsum,system_system_op_times_groups_url_page_github_cumsum,system_system_op_times_groups_url_page_github_error_code_cumsum,system_system_op_times_groups_url_page_download_cumsum,system_system_op_times_groups_url_page_download_error_code_cumsum,system_system_op_times_groups_url_sit_xxx.com_cumsum,system_system_op_times_groups_url_sit_xxx.com_error_code_cumsum,system_system_op_times_groups_url_sit_wpsdoc.xxx.com_cumsum,system_system_op_times_groups_url_sit_wpsdoc.xxx.com_error_code_cumsum,system_op_days_cumsum,system_op_days_error_code_cumsum,system_op_days_department_rd_cumsum,system_op_days_department_rd_error_code_cumsum,system_op_days_department_sales_cumsum,system_op_days_department_sales_error_code_cumsum,system_op_days_browser_version_edge_93_cumsum,system_op_days_browser_version_edge_93_error_code_cumsum,system_op_days_browser_version_chrome_90_cumsum,system_op_days_browser_version_chrome_90_error_code_cumsum,system_op_days_browser_chrome_cumsum,system_op_days_browser_chrome_error_code_cumsum,system_op_days_browser_edge_cumsum,system_op_days_browser_edge_error_code_cumsum,system_op_days_os_type_win_cumsum,system_op_days_os_type_win_error_code_cumsum,system_op_days_os_version_win10_cumsum,system_op_days_os_version_win10_error_code_cumsum,system_op_days_os_version_win7_cumsum,system_op_days_os_version_win7_error_code_cumsum,system_op_days_ip_type_内网_cumsum,system_op_days_ip_type_内网_error_code_cumsum,system_op_days_http_status_code_200_cumsum,system_op_days_op_city_深圳_cum

In [10]:
train_woe, val_woe, bins_adj = sp.binning_data_detail(train=df_train,
                        test=df_test, y=y_label, excel_path=path_output_report, var_skip=['id'],)

There are blank strings in 1 columns, which are replaced with NaN. 
 (ColumnNames: url_page)
There are 13 variables have only one binning intervals，please check the binning result. 
 (ColumnNames: system_op_days_url_page_github_error_code_cumsum, system_system_op_times_groups_department_sales_error_code_cumsum, system_op_days_log_system_transform_nHrKgKdJ1Mzt_error_code_cumsum, system_system_op_times_groups_url_page_github_error_code_cumsum, system_op_month_log_system_transform_nHrKgKdJ1Mzt_error_code_cumsum, system_system_op_times_groups_url_xxx.com/github_error_code_cumsum, system_op_month_url_page_github_error_code_cumsum, op_days_user_name_error_code_cumsum, system_system_op_times_groups_os_version_win7_error_code_cumsum, system_op_month_url_xxx.com/github_error_code_cumsum, system_system_op_times_groups_log_system_transform_nHrKgKdJ1Mzt_error_code_cumsum, system_op_days_url_xxx.com/github_error_code_cumsum, system_op_days_os_version_win7_error_code_cumsum)
sheet07.初始分箱结果在/Users/li

[INFO] converting into woe values ...


There are blank strings in 1 columns, which are replaced with NaN. 
 (ColumnNames: url_page)


Woe transformating on 47660 rows and 158 columns in 00:00:13
[INFO] converting into woe values ...


There are blank strings in 1 columns, which are replaced with NaN. 
 (ColumnNames: url_page)
sheet09.调整后分箱结果在/Users/liliangshan/workspace/python/01_datasets/ccf_system_access_risk_identification/results/01_原始数据探察_20221014.xlsx中已经存在，我们将对原文件进行覆盖


In [11]:
res_data, res_val, bins_selection = sp.select_feat_detail(train_woe, val_woe,df_test, y_label, bins_adj,exclude=['id'],
                                                iv_max_threshold=1.5,
                                                remove_negitive_coef=True,
                                                corr_threshold=0.7,
                                                #chi_threshold=0.00001,
                                                excel_path=path_output_report)

变量dayofweek_sin 的分箱结果未通过卡方检验
变量dayofweek 的分箱结果未通过卡方检验
变量op_city 的分箱结果未通过卡方检验
变量system_op_month_url_page_download_cumsum 的分箱结果未通过卡方检验
变量system_op_month_url_page_download_error_code_cumsum 的分箱结果未通过卡方检验
变量system_op_month_url_wpsdoc.xxx.com/download_cumsum 的分箱结果未通过卡方检验
变量system_op_month_url_wpsdoc.xxx.com/download_error_code_cumsum 的分箱结果未通过卡方检验
变量system_system_op_times_groups_department_rd_error_code_cumsum 的分箱结果未通过卡方检验
变量url_sit 的分箱结果未通过卡方检验


变量system_op_diff_second1_woe被加入
变量system_op_month_http_status_code_200_cumsum_woe被加入
变量op_month_ip_transform_cumsum_woe被加入
变量op_times_groups_device_num_transform_cumsum_woe被加入
变量hour_cos_woe被加入
变量system_op_days_ip_type_内网_cumsum_woe被加入
变量op_month_device_num_transform_cumsum_woe被加入
变量system_system_op_times_groups_ip_type_内网_cumsum_woe被加入
变量system_op_days_ip_type_内网_error_code_cumsum_woe被加入
变量hour_woe被加入
变量op_diff_second1_woe被加入
变量op_days_ip_transform_cumsum_woe被加入
变量url_woe被加入
变量system_system_op_times_groups_error_code_cumsum_woe被加入
变量system_system_op_times_groups_url_sit_xxx.com_cumsum_woe被加入
变量system_system_op_times_groups_department_sales_cumsum_woe被加入
变量system_op_days_op_city_深圳_cumsum_woe被加入
变量system_op_days_os_version_win10_error_code_cumsum_woe被加入
变量system_op_month_url_page_github_cumsum_woe被加入
变量system_system_op_times_groups_url_sit_xxx.com_error_code_cumsum_woe被加入
变量system_op_days_op_city_成都_cumsum_woe被加入
变量op_days_user_name_cumsum_woe被加入
变量system_op_days_browser_edge_cumsum_wo

逐步回归结果


,指标1,取值1,指标2,取值2
0,Dep. Variable:,is_risk,No. Observations:,47660.0000
1,Model:,LogisticRegression,Df Residuals:,47639.0000
2,Method:,MLE,Df Model:,20.0000
3,Date:,Thu; 20 Oct 2022,Pseudo R-squ.:,0.5698
4,Time:,14:41:57,Log-Likelihood:,-9064.3000
5,converged:,True,LL-Null:,-21071.0000
6,Covariance Type:,nonrobust,LLR p-value:,0.0000


,feat,coef,std err,z,P>|z|,[0.025,0.975]
0,intercept ...,-1.9882,0.023,-86.269,0.000,-2.033,-1.943
1,hour_cos_woe ...,0.8995,0.030,29.493,0.000,0.840,0.959
2,op_days_ip_transform_cumsum_woe ...,0.5856,0.061,9.639,0.000,0.467,0.705
3,op_days_user_name_cumsum_woe ...,1.2400,0.145,8.579,0.000,0.957,1.523
4,op_month_ip_transform_cumsum_woe ...,1.3826,0.025,54.943,0.000,1.333,1.432
5,op_month_user_name_error_code_cumsum_woe ...,0.6689,0.094,7.128,0.000,0.485,0.853
6,op_times_groups_device_num_transform_cumsum_wo...,1.3648,0.031,44.286,0.000,1.304,1.425
7,system_op_days_ip_type_内网_cumsum_woe ...,0.4070,0.040,10.178,0.000,0.329,0.485
8,system_op_days_ip_type_内网_error_code_cumsum_wo...,0.1245,0.064,1.947,0.052,-0.001,0.250
9,system_op_days_op_city_成都_cumsum_woe ...,0.0719,0.099,0.722,0.470,-0.123,0.267


sheet10.特征筛选在/Users/liliangshan/workspace/python/01_datasets/ccf_system_access_risk_identification/results/01_原始数据探察_20221014.xlsx中已经存在，我们将对原文件进行覆盖
sheet11.特征表现在/Users/liliangshan/workspace/python/01_datasets/ccf_system_access_risk_identification/results/01_原始数据探察_20221014.xlsx中已经存在，我们将对原文件进行覆盖


In [38]:
df_train = res_data
df_test = res_val

### null importance特征筛选

In [17]:
feats, categorical_feats = get_null_importance(df_train.drop(columns=[y_label,'id']),
                                               df_train[y_label], 
                                               thresholds=15)

In [18]:

object_feats = ['ip_transform', 'user_name', 'device_num_transform']

df_train = pd.concat([df_train_info[object_feats], df_train[feats], res_data], axis=1)
df_test = pd.concat([df_test[feats], res_val], axis=1)

## modeling

In [41]:

feats = df_train.columns.drop(['id', y_label])
#feats = feats.drop(categorical_feats)
#feats 

In [20]:
categorical_feats

['department', 'op_city', 'url', 'url_page']

In [30]:
feats = feats_importance.sort_values('importance', ascending=False)[:20]['name'].values

In [42]:

import time
from sklearn.metrics import roc_auc_score as auc
import lightgbm as lgb
from sklearn.model_selection import StratifiedKFold, KFold

In [43]:
params = {
    'learning_rate': 0.05,
    'boosting_type': 'gbdt',
    'objective': 'binary',
    'metric': 'auc',
    'num_leaves': 31,
    'verbose': -1,
    'seed': 2222,
    'n_jobs': -1,
    'feature_fraction': 0.8,
    'bagging_fraction': 0.9,
    'bagging_freq': 4,
    # 'min_child_weight': 10,
}

In [44]:
fold_num = 5
seeds = [2022]
oof = np.zeros(len(df_train))
importance = 0
pred_y = pd.DataFrame()
score = []
for seed in seeds:
    kf = StratifiedKFold(n_splits=fold_num, shuffle=True, random_state=seed)
    # kf = KFold(n_splits=fold_num, shuffle=True, random_state=seed)
    for fold, (train_idx, val_idx) in enumerate(kf.split(df_train[feats], df_train[y_label])):
        print('-----------', fold)
        train = lgb.Dataset(df_train.loc[train_idx, feats],
                            df_train.loc[train_idx, y_label],
                           # categorical_feature=categorical_feats
                           )
        val = lgb.Dataset(df_train.loc[val_idx, feats],
                          df_train.loc[val_idx, y_label],
                          #categorical_feature=categorical_feats
                         )
        model = lgb.train(params, train, valid_sets=[val], 
                          num_boost_round=20000, early_stopping_rounds=100)

        oof[val_idx] += model.predict(df_train.loc[val_idx, feats]) / len(seeds)
        pred_y['fold_%d_seed_%d' % (fold, seed)] = model.predict(df_test[feats])
        importance += model.feature_importance(importance_type='gain') / fold_num
        score.append(auc(df_train.loc[val_idx, y_label], model.predict(df_train.loc[val_idx, feats])))
feats_importance = pd.DataFrame()
feats_importance['name'] = feats
feats_importance['importance'] = importance
display(feats_importance.sort_values('importance', ascending=False)[:30])

df_train['oof'] = oof
display(np.mean(score), np.std(score))

score = np.mean(score)
df_test[y_label] = pred_y.mean(axis=1).values
df_test = df_test.sort_values('id').reset_index(drop=True)

sub = pd.read_csv(path_sample_submission)
sub[y_label] = df_test[y_label].values
sub.to_csv(os.path.join(path_results_jupyter,time.strftime('lgb_%Y%m%d%H%M_')+'%.5f.csv'%score), index=False)

----------- 0
[1]	valid_0's auc: 0.934797
Training until validation scores don't improve for 100 rounds
[2]	valid_0's auc: 0.935486
[3]	valid_0's auc: 0.934538
[4]	valid_0's auc: 0.936441
[5]	valid_0's auc: 0.936822
[6]	valid_0's auc: 0.938072
[7]	valid_0's auc: 0.936692
[8]	valid_0's auc: 0.936655
[9]	valid_0's auc: 0.936892
[10]	valid_0's auc: 0.936206
[11]	valid_0's auc: 0.936394
[12]	valid_0's auc: 0.936039
[13]	valid_0's auc: 0.936102
[14]	valid_0's auc: 0.936136
[15]	valid_0's auc: 0.936425
[16]	valid_0's auc: 0.935323
[17]	valid_0's auc: 0.935724
[18]	valid_0's auc: 0.935181
[19]	valid_0's auc: 0.935507
[20]	valid_0's auc: 0.935406
[21]	valid_0's auc: 0.936078
[22]	valid_0's auc: 0.935178
[23]	valid_0's auc: 0.935923
[24]	valid_0's auc: 0.936019
[25]	valid_0's auc: 0.936324
[26]	valid_0's auc: 0.936529
[27]	valid_0's auc: 0.93655
[28]	valid_0's auc: 0.936602
[29]	valid_0's auc: 0.936685
[30]	valid_0's auc: 0.936758
[31]	valid_0's auc: 0.936174
[32]	valid_0's auc: 0.93619
[33]	va

----------- 2
[1]	valid_0's auc: 0.928797
Training until validation scores don't improve for 100 rounds
[2]	valid_0's auc: 0.928649
[3]	valid_0's auc: 0.931695
[4]	valid_0's auc: 0.932426
[5]	valid_0's auc: 0.932663
[6]	valid_0's auc: 0.934474
[7]	valid_0's auc: 0.933478
[8]	valid_0's auc: 0.933566
[9]	valid_0's auc: 0.93385
[10]	valid_0's auc: 0.933441
[11]	valid_0's auc: 0.933524
[12]	valid_0's auc: 0.933293
[13]	valid_0's auc: 0.933453
[14]	valid_0's auc: 0.933413
[15]	valid_0's auc: 0.932396
[16]	valid_0's auc: 0.932331
[17]	valid_0's auc: 0.932835
[18]	valid_0's auc: 0.931962
[19]	valid_0's auc: 0.932187
[20]	valid_0's auc: 0.932528
[21]	valid_0's auc: 0.932963
[22]	valid_0's auc: 0.933255
[23]	valid_0's auc: 0.933061
[24]	valid_0's auc: 0.933123
[25]	valid_0's auc: 0.933174
[26]	valid_0's auc: 0.933193
[27]	valid_0's auc: 0.933253
[28]	valid_0's auc: 0.933336
[29]	valid_0's auc: 0.933364
[30]	valid_0's auc: 0.933551
[31]	valid_0's auc: 0.934172
[32]	valid_0's auc: 0.934191
[33]	v

----------- 3
[1]	valid_0's auc: 0.93245
Training until validation scores don't improve for 100 rounds
[2]	valid_0's auc: 0.933703
[3]	valid_0's auc: 0.934443
[4]	valid_0's auc: 0.934298
[5]	valid_0's auc: 0.935491
[6]	valid_0's auc: 0.938787
[7]	valid_0's auc: 0.938683
[8]	valid_0's auc: 0.940329
[9]	valid_0's auc: 0.940697
[10]	valid_0's auc: 0.940331
[11]	valid_0's auc: 0.940929
[12]	valid_0's auc: 0.940232
[13]	valid_0's auc: 0.940867
[14]	valid_0's auc: 0.941006
[15]	valid_0's auc: 0.940207
[16]	valid_0's auc: 0.940136
[17]	valid_0's auc: 0.940479
[18]	valid_0's auc: 0.939507
[19]	valid_0's auc: 0.939774
[20]	valid_0's auc: 0.939821
[21]	valid_0's auc: 0.940528
[22]	valid_0's auc: 0.940545
[23]	valid_0's auc: 0.940423
[24]	valid_0's auc: 0.940605
[25]	valid_0's auc: 0.94076
[26]	valid_0's auc: 0.94077
[27]	valid_0's auc: 0.940865
[28]	valid_0's auc: 0.94107
[29]	valid_0's auc: 0.941046
[30]	valid_0's auc: 0.941125
[31]	valid_0's auc: 0.940563
[32]	valid_0's auc: 0.940522
[33]	vali

,name,importance
11,system_op_diff_second1_woe,35184.785468
3,op_month_ip_transform_cumsum_woe,32095.163128
5,op_times_groups_device_num_transform_cumsum_woe,30548.735428
12,system_op_month_http_status_code_200_cumsum_woe,26392.106830
15,system_system_op_times_groups_ip_type_内网_cumsu...,14013.814972
0,hour_cos_woe,8087.973943
6,system_op_days_ip_type_内网_cumsum_woe,5327.520743
7,system_op_days_ip_type_内网_error_code_cumsum_woe,4174.567121
2,op_days_user_name_cumsum_woe,3673.432242
18,system_system_op_times_groups_url_sit_xxx.com_...,3567.464854


0.9387066041855651

0.001799127979039961

In [34]:
feats_importance.sort_values('importance', ascending=False)[:50]

,name,importance
0,system_system_op_times_groups_http_status_code...,75315.463897
2,op_month_ip_transform_cumsum,27070.662936
1,system_op_days_http_status_code_200_cumsum,26749.550276
3,op_times_groups_ip_transform_cumsum,19774.007623
4,op_diff_second1,9928.795383
6,hour,9576.983015
5,op_month_ip_transform_cumsum_woe,7084.195660
8,system_system_op_times_groups_cumsum,6787.078843
7,op_times_groups_device_num_transform_cumsum,4619.909411
9,hour_cos,3802.788354


In [35]:
feats_importance.sort_values('importance', ascending=False)[:50].name.values

array(['system_system_op_times_groups_http_status_code_200_cumsum',
       'op_month_ip_transform_cumsum',
       'system_op_days_http_status_code_200_cumsum',
       'op_times_groups_ip_transform_cumsum', 'op_diff_second1', 'hour',
       'op_month_ip_transform_cumsum_woe',
       'system_system_op_times_groups_cumsum',
       'op_times_groups_device_num_transform_cumsum', 'hour_cos',
       'system_system_op_times_groups_ip_type_内网_cumsum',
       'system_op_days_ip_type_内网_cumsum',
       'op_times_groups_user_name_cumsum',
       'op_days_device_num_transform_cumsum',
       'system_op_days_department_rd_cumsum',
       'system_system_op_times_groups_department_rd_cumsum',
       'op_times_groups',
       'system_system_op_times_groups_department_sales_cumsum',
       'system_op_days_log_system_transform_nHrKgKdJ1Mzt_cumsum',
       'system_op_month_http_status_code_200_cumsum_woe'], dtype=object)

In [ ]:
df_train_info['oof']=df_train['oof']

df_train_info[abs(df_train_info['is_risk']-df_train_info['oof'])>0.7]['device_num_transform'].value_counts()

In [ ]:
df_train_info[df_train_info['device_num_transform']=='0dV6LzVsv7pW']

In [36]:
sub_merge = pd.concat([sub,sub_sc['score']], axis=1)
sub_final = sub_merge[['id']]
sub_final['is_risk'] = 0.9423363738675891/(0.9423363738675891+0.9312)*sub_merge['is_risk'] + 0.9312/(0.9423363738675891+0.9312)*sub_merge['score']
sub_final.head()

,id,is_risk
0,0,0.035318
1,1,0.022545
2,2,0.034020
3,3,0.030858
4,4,0.024617


In [37]:

sub_final.to_csv(os.path.join(path_results_jupyter,time.strftime('lgb_sc_%Y%m%d%H%M')+'.csv'), index=False)

In [ ]:
sub_final.tail()